# Introduction to [PowerSystems.jl](https://github.com/NREL-SIIP/PowerSystems.jl)

**Originally Contributed by**: Clayton Barrows and Jose Daniel Lara

## Introduction

This notebook is intended to show a power system data specification framework that exploits the
capabilities of Julia to improve performance and allow modelers to develop modular software
to create problems with different complexities and enable large scale analysis. The
[PowerSystems documentation](https://nrel-siip.github.io/PowerSystems.jl/stable/) is also
an excellent resource.

### Objective
PowerSystems.jl provides a type specification for bulk power system data.
The objective is to exploit Julia's integration of dynamic types to enable efficient data
handling and enable functional dispatch in modeling and analysis applications
As explained in Julia's documentation:

"Julia’s type system is dynamic, but gains some of the advantages of static type systems
by making it possible to indicate that certain values are of specific types. This can be
of great assistance in generating efficient code, but even more significantly, it allows
method dispatch on the types of function arguments to be deeply integrated with the language."

For more details on Julia types, refer to the [documentation](https://docs.julialang.org/en/v1/)


## Environment and packages

PowerSystems.jl relies on a framework for data handling established in
[InfrastructureSystems.jl](https://github.com/NREL-SIIP/InfrastructureSystems.jl).
Users of PowerSystems.jl should not need to interact directly with InfrastructureSystems.jl.
However, it's worth recognizing that InfrastructureSystems provides much of the back end
code for managing and accessing data, especially time series data.

In [1]:
using SIIPExamples;
using PowerSystems;

Normally, I'd add the following two lines to configure logging behavior, but something about
Literate.jl makes this fail, so these examples only work with the default log configuration.
```julia
using Logging
logger = configure_logging(console_level = Logging.Error, file_level = Logging.Info, filename = "ex.log")
````

## Types in PowerSystems
PowerSystems.jl provides a type hierarchy for specifying power system data. Data that
describes infrastructure components is held in `struct`s. For example, a `Bus` is defined
as follows with fields for the parameters required to describe a bus (along with an
`internal` field used by InfrastructureSystems to improve the efficiency of handling data).

In [2]:
print_struct(Bus)

mutable struct PowerSystems.Bus
    number::Int64
    name::String
    bustype::Union{Nothing, PowerSystems.BusTypesModule.BusTypes}
    angle::Union{Nothing, Float64}
    magnitude::Union{Nothing, Float64}
    voltage_limits::Union{Nothing, NamedTuple{(:min, :max), Tuple{Float64, Float64}}}
    base_voltage::Union{Nothing, Float64}
    area::Union{Nothing, PowerSystems.Area}
    load_zone::Union{Nothing, PowerSystems.LoadZone}
    ext::Dict{String, Any}
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


### Type Hierarchy
PowerSystems is intended to organize data containers by the behavior of the devices that
the data represents. To that end, a type hierarchy has been defined with several levels of
abstract types starting with `InfrastructureSystemsType`. There are a bunch of subtypes of
`InfrastructureSystemsType`, but the important ones to know about are:
- `Component`: includes all elements of power system data
  - `Topology`: includes non physical elements describing network connectivity
  - `Service`: includes descriptions of system requirements (other than energy balance)
  - `Device`: includes descriptions of all the physical devices in a power system
- `InfrastructureSystems.DeviceParameter`: includes structs that hold data describing the
 dynamic, or economic capabilities of `Device`.
- `TimeSeriesData`: Includes all time series types
  - `Forecast`: includes structs to define time series of forecasted data where multiple
values can represent each time stamp
  - `StaticTimeSeries`: includes structs to define time series with a single value for each
time stamp
- `System`: collects all of the `Component`s

In [3]:
print_tree(PowerSystems.IS.InfrastructureSystemsType)

InfrastructureSystemsType
├─ DeviceParameter
│  ├─ ActivePowerControl
│  │  ├─ ActivePowerDroop
│  │  ├─ ActivePowerPI
│  │  ├─ ActiveRenewableControllerAB
│  │  └─ VirtualInertia
│  ├─ DynamicComponent
│  │  ├─ DynamicGeneratorComponent
│  │  │  ├─ AVR
│  │  │  │  ├─ AVRFixed
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ AVRSimple
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ AVRTypeI
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ AVRTypeII
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ ESAC1A
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ ESAC6A
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ ESDC1A
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ ESDC2A
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ ESST1A
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ ESST4B
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ EXAC1
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ EXAC1A
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ EXAC2
│  │  │  │  │  ⋮
│  │  │  │  │  
│  │  │  │  ├─ EXPIC1
│

### `TimeSeriesData`
[_Read the Docs!_](https://nrel-siip.github.io/PowerSystems.jl/stable/modeler_guide/time_series/)
Every `Component` has a `time_series_container::InfrastructureSystems.TimeSeriesContainer`
field. `TimeSeriesData` are used to hold time series information that describes the
temporally dependent data of fields within the same struct. For example, the
`ThermalStandard.time_series_container` field can
describe other fields in the struct (`available`, `activepower`, `reactivepower`).

`TimeSeriesData`s themselves can take the form of the following:

In [4]:
print_tree(TimeSeriesData)

TimeSeriesData
├─ Forecast
│  ├─ AbstractDeterministic
│  │  ├─ Deterministic
│  │  └─ DeterministicSingleTimeSeries
│  ├─ Probabilistic
│  └─ Scenarios
└─ StaticTimeSeries
   └─ SingleTimeSeries


In each case, the time series contains fields for `scaling_factor_multiplier` and `data`
to identify the details of  th `Component` field that the time series describes, and the
time series `data`. For example: we commonly want to use a time series to
describe the maximum active power capability of a renewable generator. In this case, we
can create a `SingleTimeSeries` with a `TimeArray` and an accessor function to the
maximum active power field in the struct describing the generator. In this way, we can
store a scaling factor time series that will get multiplied by the maximum active power
rather than the magnitudes of the maximum active power time series.

In [5]:
print_struct(Deterministic)

mutable struct InfrastructureSystems.Deterministic
    name::String
    data::Union{DataStructures.SortedDict{Dates.DateTime, Vector{Vector{Tuple{Float64, Float64}}}}, DataStructures.SortedDict{Dates.DateTime, Vector{Float64}}, DataStructures.SortedDict{Dates.DateTime, Vector{Tuple{Float64, Float64}}}}
    resolution::Dates.Period
    scaling_factor_multiplier::Union{Nothing, Function}
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


Examples of how to create and add time series to system can be found in the
[Add Time Series Example](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/2_PowerSystems_examples/05_add_forecasts.ipynb)

### System
The `System` object collects all of the individual components into a single struct along
with some metadata about the system itself (e.g. `base_power`)

In [6]:
print_struct(System)

mutable struct PowerSystems.System
    data::InfrastructureSystems.SystemData
    frequency::Float64
    bus_numbers::Set{Int64}
    runchecks::Base.RefValue{Bool}
    units_settings::InfrastructureSystems.SystemUnitsSettings
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


## Basic example
PowerSystems contains a few basic data files (mostly for testing and demonstration).

In [7]:
BASE_DIR = abspath(joinpath(dirname(Base.find_package("PowerSystems")), ".."))
include(joinpath(BASE_DIR, "test", "data_5bus_pu.jl")) #.jl file containing 5-bus system data
nodes_5 = nodes5() # function to create 5-bus buses

5-element Vector{PowerSystems.Bus}:
 Bus(1, nodeA, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(2, nodeB, PowerSystems.BusTypesModule.BusTypes.PQ = 2, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(3, nodeC, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(4, nodeD, PowerSystems.BusTypesModule.BusTypes.REF = 4, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(5, nodeE, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())

### Create a `System`

In [8]:
sys = System(
    100.0,
    nodes_5,
    vcat(thermal_generators5(nodes_5), renewable_generators5(nodes_5)),
    loads5(nodes_5),
    branches5(nodes_5),
)

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ SYSTEM_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 28          │
└───────────────────┴─────────────┘

Static Components
┌───────────────────┬───────┬────────────────────────┬───────────────┐
│ Type              │ Count │ Has Static Time Series │ Has Forecasts │
├───────────────────┼───────┼────────────────────────┼───────────────┤
│ Arc               │ 6     │ false                  │ false         │
│ Bus               │ 5     │ false                  │ false         │
│ Line              │ 6     │ false                  │ false         │
│ PowerLoad         │ 3     │ false                  │ false         │
│ RenewableDispatch │ 3     │ false                  │ false         │
│ ThermalStandard   │ 5     │ false                  │ false         │
└───────────────────┴───────┴────────────────────────┴───────────────┘

### Accessing `System` Data
PowerSystems provides functional interfaces to all data. The following examples outline
the intended approach to accessing data expressed using PowerSystems.

PowerSystems enforces unique `name` fields between components of a particular concrete type.
So, in order to retrieve a specific component, the user must specify the type of the component
along with the name and system

#### Accessing components

In [9]:
@show get_component(Bus, sys, "nodeA")
@show get_component(Line, sys, "1")

get_component(Bus, sys, "nodeA") = Bus(1, nodeA, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
get_component(Line, sys, "1") = Line(1, true, 0.0, 0.0, Arc(Bus(1, nodeA, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}()), Bus(2, nodeB, PowerSystems.BusTypesModule.BusTypes.PQ = 2, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())), 0.00281, 0.0281, (from = 0.00356, to = 0.00356), 2.0, (min = -0.7, max = 0.7), PowerSystems.Service[], Dict{String, Any}())


1 (PowerSystems.Line):
   name: 1
   available: true
   active_power_flow: 0.0
   reactive_power_flow: 0.0
   arc: nodeA -> nodeB (PowerSystems.Arc)
   r: 0.00281
   x: 0.0281
   b: (from = 0.00356, to = 0.00356)
   rate: 2.0
   angle_limits: (min = -0.7, max = 0.7)
   services: 0-element Vector{PowerSystems.Service}
   ext: Dict{String, Any}()
   time_series_container: InfrastructureSystems.TimeSeriesContainer: 0
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: InfrastructureSystems.UnitSystemModule.UnitSystem.SYSTEM_BASE = 0

Similarly, you can access all the components of a particular type: *note: the return type
of get_components is a `FlattenIteratorWrapper`, so call `collect` to get an `Array`

In [10]:
get_components(Bus, sys) |> collect

5-element Vector{PowerSystems.Bus}:
 Bus(1, nodeA, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(3, nodeC, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(5, nodeE, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(2, nodeB, PowerSystems.BusTypesModule.BusTypes.PQ = 2, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())
 Bus(4, nodeD, PowerSystems.BusTypesModule.BusTypes.REF = 4, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())

`get_components` also works on abstract types:

In [11]:
get_components(Branch, sys) |> collect

6-element Vector{PowerSystems.Branch}:
 Line(4, true, 0.0, 0.0, Arc(Bus(2, nodeB, PowerSystems.BusTypesModule.BusTypes.PQ = 2, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}()), Bus(3, nodeC, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())), 0.00108, 0.0108, (from = 0.00926, to = 0.00926), 11.148, (min = -0.7, max = 0.7), PowerSystems.Service[], Dict{String, Any}())
 Line(1, true, 0.0, 0.0, Arc(Bus(1, nodeA, PowerSystems.BusTypesModule.BusTypes.PV = 3, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}()), Bus(2, nodeB, PowerSystems.BusTypesModule.BusTypes.PQ = 2, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String, Any}())), 0.00281, 0.0281, (from = 0.00356, to = 0.00356), 2.0, (min = -0.7, max = 0.7), PowerSystems.Service[], Dict{String, Any}())
 Line(5, true, 0.0, 0.0, Arc(Bus(3, nodeC, PowerSystems.BusTypesModule.BusTypes.PV = 3

The fields within a component can be accessed using the `get_*` functions:
*It's highly recommended that users avoid using the `.` to access fields since we make no
guarantees on the stability field names and locations. We do however promise to keep the
accessor functions stable.*

In [12]:
bus1 = get_component(Bus, sys, "nodeA")
@show get_name(bus1);
@show get_magnitude(bus1);

get_name(bus1) = "nodeA"
get_magnitude(bus1) = 1.0


#### Accessing `TimeSeries`

First we need to add some time series to the `System`

In [13]:
loads = collect(get_components(PowerLoad, sys))
for (l, ts) in zip(loads, load_timeseries_DA[2])
    add_time_series!(
        sys,
        l,
        Deterministic(
            "activepower",
            Dict(TimeSeries.timestamp(load_timeseries_DA[2][1])[1] => ts),
        ),
    )
end

If we want to access a specific time series for a specific component, we need to specify:
 - time series type
 - `component`
 - initial_time
 - label

We can find the initial time of all the time series in the system:

In [14]:
get_forecast_initial_times(sys)

1-element Vector{Dates.DateTime}:
 2024-01-02T00:00:00

We can find the names of all time series attached to a component:

In [15]:
ts_names = get_time_series_names(Deterministic, loads[1])

1-element Vector{String}:
 "activepower"

We can access a specific time series for a specific component:

In [16]:
ta = get_time_series_array(Deterministic, loads[1], ts_names[1])

24×1 TimeSeries.TimeArray{Float64, 1, Dates.DateTime, SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int64}}, true}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00
│                     │ A      │
├─────────────────────┼────────┤
│ 2024-01-02T00:00:00 │ 0.8901 │
│ 2024-01-02T01:00:00 │ 0.7728 │
│ 2024-01-02T02:00:00 │ 0.7634 │
│ 2024-01-02T03:00:00 │ 0.6964 │
│ 2024-01-02T04:00:00 │ 0.7597 │
│ 2024-01-02T05:00:00 │ 0.6844 │
│ 2024-01-02T06:00:00 │ 0.7188 │
│ 2024-01-02T07:00:00 │ 0.7945 │
│ ⋮                   │ ⋮      │
│ 2024-01-02T17:00:00 │ 0.9164 │
│ 2024-01-02T18:00:00 │ 1.0162 │
│ 2024-01-02T19:00:00 │ 1.0919 │
│ 2024-01-02T20:00:00 │ 1.0407 │
│ 2024-01-02T21:00:00 │ 1.0151 │
│ 2024-01-02T22:00:00 │ 0.9756 │
│ 2024-01-02T23:00:00 │ 0.8975 │

Or, we can just get the values of the time series:

In [17]:
ts = get_time_series_values(Deterministic, loads[1], ts_names[1])

24-element view(::Vector{Float64}, 1:24) with eltype Float64:
 0.8900534951404266
 0.7727964273707415
 0.7633821930260857
 0.6963567083409643
 0.7597162088709701
 0.6843790958436067
 0.7188038207731461
 0.7944910514718007
 0.8349778461938684
 0.803120154154377
 ⋮
 0.9062616868035502
 0.92274505678347
 0.9164149163200961
 1.016151080389699
 1.0919104343657873
 1.040720873359067
 1.0150733148097077
 0.9755756388186618
 0.8974771634770712

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*